In [1]:
from threading import Thread

In [ ]:
class ant_colony:
    
    class ant(Thread):
        
        def __init__(self, init_location, possible_locations, pheromone_map, distance_callback, alpha, beta, first_pass=False):
            
            Thread.__init__(self)
            
            self.init_location = init_location
            self.possible_locations = possible_locations
            self.route = []
            self.distance_traveled = 0.0
            self.location = init_location
            self.pheromone_map = pheromone_map
            self.distance_callback = distance_callback
            self.alpha = alpha
            self.beta = beta
            self.first_pass = first_pass
            
            # append start location to route, before doing random walk
            
            self._update_route(init_location)
            
            self.tour_complete = False
        
        
        def run(self):
            
            while self.possible_locations:
                
                next = self._pick_path()
                
                self._traverse(self.location, next)
            
            self.tour_complete = True
            
        
        def _pick_path(self):
            
            
            # on the first pass (no pheromones), then we can just choice() to find the next one
            
            if self.first_pass:
                
                import random
                
                return random.choice(self.possible_locations)
            
            attractiveness = dict()
            
            sum_total = 0.0
            
            # for each possible location, find its attractiveness (it´s  (pheromone amount)*1/distance[tau * eta, from the algorithm ])
            # sum all attrativeness amounts for calculating probability of each route in the next step
            
            for possible_next_location in self.possible_locations:
                
                # note: do all calculations as float
                
                pheromone_amount = float(self.pheromone_map[self.location][posiible_next_location])
                
                distance = float(self.distance_callback(self.location, possible_next_location))
                
                # tau^alpha * eta^beta
                
                attractiveness[possible_next_location] = pow(pheromone_amount, self.alpha) * pow(1/distance, self.beta)
                
                sum_total += attractiveness[possible_next_location]
                
            # it is possible to have small values for pheromone amount / distance, such that with rounding errors this is equal to zero 
            # rare, but handle when it happens 
            
            if sum_total == 0.0:
                
                def next_up(x):
                    
                    import math
                    import struct
                    
                    if math.isnan(x) or (math.isnan(x) and x > 0):
                        
                        return x
                    
                    # 0.0 and -0.0 both map to the smallest +ve float
                    
                    if x == 0.0:
                        
                        x = 0.0
                        
                    n = struct.unpack('<q', struct.pack('<d', x))[0]
                    
                    if n >=0:
                        
                        n += 1
                        
                    else:
                        
                        n -= 1
                    
                    return struct.unpack('<d', struct.pack('<q', n))[0]
                
                
                for key in attractiveness:
                    
                    attractiveness[key] = next_up(attractiveness[key])
                
                sum_total = next_up(sum_total)
                
            
            # randomly choose the next path
            
            import random
            
            toss = random.random()
            
            cummulative = 0
            
            for possible_next_location in attractiveness:
                
                weight = (attractiveness[possible_next_location] / sum_total)
                
                if toss <= weight + cummulative:
                    
                    return possible_next_location
                
                cummulative += weight
                
        
        
        def _traverse(self, start, end):
            
            self._update_route(end)
            self._update_distance_traveled(start, end)
            self.location = end
        
        
        def _update_route(self, new):
            
            self.route.append(new)
            self.possible_locations.remove(new)
        
        def _update_distance_traveled(self, start, end):
            
            self.distance_traveled += float(self.distance_callback(start, end))
            
        def get_route(self):
            
            if self.tour_complete:
                
                return self.route
            
            return None
        
        def get_distance_traveled(self):
            
            if self.tour_complete:
                
                return self.distance_traveled
            
            return None                       # 158 lines
            
                
                
                
            